In [1]:
from clasifier import *

In [2]:
with open('constitucion_colombiana.txt', 'r') as f:
    text = f.readlines()

    f.close()
    
for line in text:
    if line == '\n':
        text.remove(line)

In [3]:
clean_text = text_remove_nl(text)
# Drop the title "Constitucion Politica de Colombia"
clean_text = clean_text[1:len(text)]

In [4]:
chapters = join_labels(clean_text, label = 'TITULO')
# for key in chapters:
#     print(key, chapters[key]['name'], chapters[key]['content'][:2])

print('\n------------------\n')
test_key = 'TITULO II\n'
test_text = chapters[test_key]['content']
print(test_text[-2])
print('\n------------------\n')

title_2 = chapters = join_labels(test_text, label = 'CAPITULO', name_importance = True)
for key in title_2:
    print(key,title_2[key]['content'], end='\n\n')


------------------

8. Proteger los recursos culturales y naturales del país y velar por la conservación de un ambiente sano,


------------------

CAPITULO 1
 ['Artículo 11. El derecho a la vida es inviolable. No habrá pena de muerte.\n', 'Artículo 12. Nadie será sometido a desaparición forzada, a torturas ni a tratos o penas crueles, inhumanos o degradantes.\n', 'Artículo 13. Todas las personas nacen libres e iguales ante la ley, recibirán la misma protección y trato de las autoridades y gozarán de los mismos derechos, libertades y oportunidades sin ninguna discriminación por razones de sexo, raza, origen nacional o familiar, lengua, religión, opinión política o filosófica.\n', 'El Estado promoverá las condiciones para que la igualdad sea real y efectiva y adoptara medidas en favor de grupos discriminados o marginados.\n', 'El Estado protegerá especialmente a aquellas personas que por su condición económica, física o mental, se encuentren en circunstancia de debilidad manifiesta y s